# 5.2.2 MovieLensのデータ概要

In [1]:
import pandas as pd

In [2]:
m_cols = ["movie_id", "title", "genre"]
movies = pd.read_csv("../data/ml-10M100K/movies.dat", names=m_cols, sep="::", encoding="latin-1", engine="python")
movies["genre"] = movies.genre.apply(lambda x: x.split("|"))
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]


In [3]:
t_cols = ["user_id", "movie_id", "tag", "timestamp"]
user_tagged_movies = pd.read_csv("../data/ml-10M100K/tags.dat", names=t_cols, sep="::", engine="python")
user_tagged_movies["tag"] = user_tagged_movies["tag"].str.lower()
user_tagged_movies.head()


,user_id,movie_id,tag,timestamp
0,15,4973,excellent!,1215184630
1,20,1747,politics,1188263867
2,20,1747,satire,1188263867
3,20,2424,chick flick 212,1188263835
4,20,2424,hanks,1188263835


In [4]:
movie_tags = user_tagged_movies.groupby("movie_id").agg({"tag":list})
movies = movies.merge(movie_tags, on='movie_id', how="left")
movies.head()

,movie_id,title,genre,tag
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]","[pixar, pixar, pixar, animation, pixar, animat..."
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]","[for children, game, animals, joe johnston, ro..."
2,3,Grumpier Old Men (1995),"[Comedy, Romance]","[funniest movies, comedinha de velhinhos engra..."
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",[girl movie]
4,5,Father of the Bride Part II (1995),[Comedy],"[steve martin, pregnancy, remake, steve martin..."


In [5]:
r_cols = ["user_id", "movie_id", "rating", "timestamp"]
ratings = pd.read_csv("../data/ml-10M100K/ratings.dat", names=r_cols, sep="::", engine="python")
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,122,5.0,838985046
1,1,185,5.0,838983525
2,1,231,5.0,838983392
3,1,292,5.0,838983421
4,1,316,5.0,838983392


In [6]:
valid_user_ids = sorted(ratings["user_id"].unique())[:1000]
ratings = ratings[ratings["user_id"].isin(valid_user_ids)]

movielens = ratings.merge(movies, on="movie_id")
movielens.head()

,user_id,movie_id,rating,timestamp,title,genre,tag
0,1,122,5.0,838985046,Boomerang (1992),"[Comedy, Romance]","[dating, nudity (topless - brief), can't remem..."
1,139,122,3.0,974302621,Boomerang (1992),"[Comedy, Romance]","[dating, nudity (topless - brief), can't remem..."
2,149,122,2.5,1112342322,Boomerang (1992),"[Comedy, Romance]","[dating, nudity (topless - brief), can't remem..."
3,182,122,3.0,943458784,Boomerang (1992),"[Comedy, Romance]","[dating, nudity (topless - brief), can't remem..."
4,215,122,4.5,1102493547,Boomerang (1992),"[Comedy, Romance]","[dating, nudity (topless - brief), can't remem..."


# 5.2.3 評価方法

In [7]:
movielens["timestamp_rank"] = movielens.groupby("user_id")["timestamp"].rank(ascending=False, method="first")
movielens_train = movielens[movielens["timestamp_rank"] > 5]
movielens_test = movielens[movielens["timestamp_rank"] <= 5]

# 5.2.4 統一フォーマットによる計算

In [8]:
import sys
sys.path.append("../")

from utils.data_loader import DataLoader

In [9]:
data_loader = DataLoader()
movielens = data_loader.load()